## Model Checkpoints
After running the notebook `DMM-Setup.ipynb`, the folder `chkpt-ipython` should now be filled with checkpoint files.

In [9]:
import os
import numpy as np
from utils.misc import loadHDF5
assert os.path.exists('chkpt-ipython/'),'Run the notebook DMM-Setup.ipynb first'

In [1]:
ls chkpt-ipython/*ep-40*

chkpt-ipython/DMM_lr-0_0008-dh-40-ds-2-nl-relu-bs-200-ep-40-rs-80-rd-0_1-infm-R-tl-2-el-2-ar-2_0-use_p-approx-rc-lstm-uid-EP40-optParams.npz
chkpt-ipython/DMM_lr-0_0008-dh-40-ds-2-nl-relu-bs-200-ep-40-rs-80-rd-0_1-infm-R-tl-2-el-2-ar-2_0-use_p-approx-rc-lstm-uid-EP40-params.npz
chkpt-ipython/DMM_lr-0_0008-dh-40-ds-2-nl-relu-bs-200-ep-40-rs-80-rd-0_1-infm-R-tl-2-el-2-ar-2_0-use_p-approx-rc-lstm-uid-EP40-stats.h5


* There are three different file types of note: 
    * `prefix-EP<#>-params.npz` : Contains the parameters of the model (weight matrices)     
    * `prefix-EP<#>-optParams.npz` : Contains the parameters corresponding to the optimization process (for ADAM, it contains the momentum variables for the model parameters)
    * With the above two files, along with the setup for the computational flow graph in `model/dmm.py`, we can restore a model to its setting at Epoch `<#>`
    * `prefix-EP<#>-stats.h5` : Contains statistics (such as the bound evaluated on validation data) calculated at training time. 
* Lets look at each kind of file and see whats in it. 

### Probing Model Parameters
The model parameters can be loaded and investigated as follows. The save file contains a `dict` whose key is the name of the paramter as created in `model_th/dmm.py`.

In [3]:
opt_params = np.load('./chkpt-ipython/DMM_lr-0_0008-dh-40-ds-2-nl-relu-bs-200-ep-40-rs-80-rd-0_1-infm-R-tl-2-el-2-ar-2_0-use_p-approx-rc-lstm-uid-EP40-optParams.npz')
for k in opt_params: 
    print k, opt_params[k].shape

opt_v_q_W_st (2, 80)
opt_i ()
opt_m_q_W_st (2, 80)
opt_m_p_emis_b_out (6,)
opt_v_p_trans_W_0 (2, 40)
opt_v_p_trans_W_1 (40, 40)
opt_m_q_b_cov (2,)
opt_m_q_W_input_0 (3, 80)
opt_v_p_emis_b_1 (40,)
opt_v_p_emis_b_out (6,)
opt_v_p_trans_b_1 (40,)
opt_v_q_W_cov (80, 2)
opt_m_W_lstm_r (80, 320)
opt_v_b_lstm_r (320,)
opt_m_p_trans_W_1 (40, 40)
opt_v_p_trans_b_cov (2,)
opt_m_q_W_cov (80, 2)
opt_v_q_b_mu (2,)
opt_m_p_trans_W_mu (40, 2)
opt_m_p_trans_W_0 (2, 40)
opt_v_q_b_input_0 (80,)
opt_m_p_emis_W_0 (2, 40)
opt_m_p_emis_W_1 (40, 40)
opt_v_p_trans_W_mu (40, 2)
opt_v_p_trans_b_mu (2,)
opt_v_q_b_cov (2,)
opt_m_b_lstm_r (320,)
opt_v_p_trans_W_cov (40, 2)
opt_m_p_trans_W_cov (40, 2)
opt_v_p_emis_W_out (40, 6)
opt_v_W_lstm_r (80, 320)
opt_v_q_b_st (80,)
opt_v_p_emis_W_1 (40, 40)
opt_v_p_emis_W_0 (2, 40)
opt_m_p_trans_b_cov (2,)
opt_v_p_emis_b_0 (40,)
opt_m_p_trans_b_0 (40,)
opt_m_p_trans_b_1 (40,)
opt_m_q_W_mu (80, 2)
opt_v_q_W_mu (80, 2)
opt_m_p_trans_b_mu (2,)
opt_v_q_W_input_0 (3, 80)
opt_m_p_e

In [4]:
opt_params = np.load('./chkpt-ipython/DMM_lr-0_0008-dh-40-ds-2-nl-relu-bs-200-ep-40-rs-80-rd-0_1-infm-R-tl-2-el-2-ar-2_0-use_p-approx-rc-lstm-uid-EP40-params.npz')
for k in opt_params: 
    print k, opt_params[k].shape

p_trans_b_cov (2,)
q_W_cov (80, 2)
p_emis_W_0 (2, 40)
p_emis_W_1 (40, 40)
q_b_st (80,)
q_W_input_0 (3, 80)
q_W_st (2, 80)
q_b_input_0 (80,)
b_lstm_r (320,)
p_emis_W_out (40, 6)
p_trans_b_0 (40,)
p_trans_b_1 (40,)
p_trans_b_mu (2,)
q_b_mu (2,)
lr ()
anneal ()
p_emis_b_1 (40,)
p_emis_b_0 (40,)
p_trans_W_mu (40, 2)
update_ctr ()
W_lstm_r (80, 320)
q_W_mu (80, 2)
U_lstm_r (80, 320)
q_b_cov (2,)
p_emis_b_out (6,)
p_trans_W_cov (40, 2)
p_trans_W_1 (40, 40)
p_trans_W_0 (2, 40)


In [5]:
import glob, os, sys, time
sys.path.append('../')
from utils.misc import getConfigFile, readPickle, displayTime
start_time = time.time()
from   model_th.dmm import DMM
import model_th.learning as DMM_learn
import model_th.evaluate as DMM_evaluate
displayTime('importing DMM',start_time, time.time())

		< importing DMM > took  2.24747085571   seconds


### Reloading models from a checkpoint
To reload a model using the saved checkpoint files, we need: 
    * The hyperparameters to rebuild the model 
    * The settings for the weight matrices that comprise the model parameters. 
* There is a `prefix` which defines the identifier for a checkpoint. The prefix is set at model creation using the unique identifier for the model. 

In [13]:
#This is the prefix we will use
DIR    = './chkpt-ipython/'
prefix = 'DMM_lr-0_0008-dh-40-ds-2-nl-relu-bs-200-ep-40-rs-80-rd-0_1-infm-R-tl-2-el-2-ar-2_0-use_p-approx-rc-lstm-uid'
pfile  =  os.path.join(DIR,prefix+'-config.pkl')
print os.path.exists(pfile)

False


In [12]:
#The hyperparameters are saved in a pickle file - lets load them here
params = readPickle(pfile)[0]

IOError: [Errno 2] No such file or directory: './chkpt-ipython/DMM_lr-0_0008-dh-40-ds-2-nl-relu-bs-200-ep-40-rs-80-rd-0_1-infm-R-tl-2-el-2-ar-2_0-use_p-approx-rc-lstm-uid-config.pkl'

In [7]:
EP     = '-EP40'
#Point the model to the saved pickle file which contains the hyperparameters of the model as well as 
#the epoch specific reloadfile

reloadFile  =  os.path.join(DIR,prefix+EP+'-params.npz')
dmm_reloaded  = DMM(params, paramFile = pfile, reloadFile = reloadFile)

./chkpt-ipython/DMM_lr-0_0008-dh-40-ds-2-nl-relu-bs-200-ep-40-rs-80-rd-0_1-infm-R-tl-2-el-2-ar-2_0-use_p-approx-rc-lstm-uid-EP40-config.pkl
